In [ ]:
# wordcloud and boost specific words?
# statsmodels, ordinary least squares, pvalue


In [2]:
import pandas as pd
import numpy as np
from string import punctuation
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE


In [6]:
fname = "../data/S1_8Sharktankpitchesdeals.csv"
data = pd.read_csv(fname)
#valuation is a component of the amoutn asked for and the stake offered therefor doesn't need to be included in the machine learning
data = data.drop(columns=["Valuation"])
data = data[data.Deal_Status != 0]
data

,Season_Epi_code,Pitched_Business_Identifier,Amount_Asked_For,Exchange_For_Stake,Gender,Category,Pitched_Business_Desc,Deal_Status,Kevin O'Leary,Lori Greiner,Mark Cuban,Robert Herjavec,Daymond John,Barbara Corcoran,Others,Deal_Shark 1,Deal_Shark 2,Deal_Shark 3,Deal_Shark 4,Deal_Shark 5
0,101,Ava the Elephant,50000,0.15,Female,Health / Wellness,"(Emmy the Elephant during show, trademarked a...",1,0,0,0,0,0,1,0,Barbara Corcoran,NaN,NaN,NaN,NaN
3,101,Mr. Tod's Pie Factory,460000,0.10,Male,Food and Beverage,a pie company,1,0,0,0,0,1,1,0,Barbara Corcoran,Daymond John,NaN,NaN,NaN
5,102,A Perfect Pear,500000,0.15,Female,Food and Beverage,a gourmet food business . a gourmet food business,1,0,0,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
7,102,Classroom Jams,250000,0.10,Male,Children / Education,a musical way to teach students Shakespeare . ...,1,1,0,0,1,1,1,0,Barbara Corcoran,Daymond John,Kevin O'Leary,Robert Herjavec,NaN
12,103,Stress Free Kids,250000,0.20,Female,Children / Education,a line of books that use stress relieving tech...,1,0,0,0,0,0,1,0,Barbara Corcoran,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,824,Rumi Spice,250000,0.05,Mixed,Food and Beverage,saffron imported from Afghanistan . Rumi Spice...,1,0,0,1,0,0,0,0,Mark Cuban,NaN,NaN,NaN,NaN
698,824,Thompson Tee,700000,0.07,Male,Fashion / Beauty,undershirts designed to block underarm sweat ....,1,0,0,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
699,824,Wallet Buckle,500000,0.10,Male,Fashion / Beauty,a belt buckle that holds credit cards . The Wa...,1,0,0,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
700,826,Bridal Buddy,75000,0.10,Female,Fashion / Beauty,a functional slip worn under a wedding gown th...,1,1,1,0,0,0,0,0,Kevin O'Leary,Lori Greiner,NaN,NaN,NaN


In [8]:
data2 = pd.DataFrame(columns=data.columns)
offset = 0;

for i, j in data.iterrows():
    data2 = data2.append(data.loc[i].to_dict(), ignore_index=True)
    
    if not pd.isna(data["Deal_Shark 2"][i]):
        data2 = data2.append(data.loc[i].to_dict(), ignore_index=True)
        offset+=1
        data2["Deal_Shark 1"][offset+i] = data.loc[i]["Deal_Shark 2"]
    if not pd.isna(data["Deal_Shark 3"][i]):
        data2 = data2.append(data.loc[i].to_dict(), ignore_index=True)
        offset+=1
        data2["Deal_Shark 1"][offset+i] = data.loc[i]["Deal_Shark 3"]
    if not pd.isna(data["Deal_Shark 4"][i]):
        data2 = data2.append(data.loc[i].to_dict(), ignore_index=True)
        offset+=1
        data2["Deal_Shark 1"][offset+i] = data.loc[i]["Deal_Shark 4"]
    if not pd.isna(data["Deal_Shark 5"][i]):
        data2 = data2.append(data.loc[i].to_dict(), ignore_index=True)
        offset+=1
        data2["Deal_Shark 1"][offset+i] = data.loc[i]["Deal_Shark 5"]


data2 = data2.drop(columns=["Deal_Shark 2","Deal_Shark 3","Deal_Shark 4","Deal_Shark 5"])
data2 = data2[data2.Deal_Status != 0]
data

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\

,Season_Epi_code,Pitched_Business_Identifier,Amount_Asked_For,Exchange_For_Stake,Gender,Category,Pitched_Business_Desc,Deal_Status,Kevin O'Leary,Lori Greiner,Mark Cuban,Robert Herjavec,Daymond John,Barbara Corcoran,Others,Deal_Shark 1,Deal_Shark 2,Deal_Shark 3,Deal_Shark 4,Deal_Shark 5
0,101,Ava the Elephant,50000,0.15,Female,Health / Wellness,"(Emmy the Elephant during show, trademarked a...",1,0,0,0,0,0,1,0,Barbara Corcoran,NaN,NaN,NaN,NaN
3,101,Mr. Tod's Pie Factory,460000,0.10,Male,Food and Beverage,a pie company,1,0,0,0,0,1,1,0,Barbara Corcoran,Daymond John,NaN,NaN,NaN
5,102,A Perfect Pear,500000,0.15,Female,Food and Beverage,a gourmet food business . a gourmet food business,1,0,0,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
7,102,Classroom Jams,250000,0.10,Male,Children / Education,a musical way to teach students Shakespeare . ...,1,1,0,0,1,1,1,0,Barbara Corcoran,Daymond John,Kevin O'Leary,Robert Herjavec,NaN
12,103,Stress Free Kids,250000,0.20,Female,Children / Education,a line of books that use stress relieving tech...,1,0,0,0,0,0,1,0,Barbara Corcoran,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,824,Rumi Spice,250000,0.05,Mixed,Food and Beverage,saffron imported from Afghanistan . Rumi Spice...,1,0,0,1,0,0,0,0,Mark Cuban,NaN,NaN,NaN,NaN
698,824,Thompson Tee,700000,0.07,Male,Fashion / Beauty,undershirts designed to block underarm sweat ....,1,0,0,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
699,824,Wallet Buckle,500000,0.10,Male,Fashion / Beauty,a belt buckle that holds credit cards . The Wa...,1,0,0,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
700,826,Bridal Buddy,75000,0.10,Female,Fashion / Beauty,a functional slip worn under a wedding gown th...,1,1,1,0,0,0,0,0,Kevin O'Leary,Lori Greiner,NaN,NaN,NaN


In [9]:
data = data.infer_objects()
data2 = data2.infer_objects()


In [10]:
# Choose model output

# "Deal_Status"
# "Deal_Shark 1"

# "Kevin O'Leary"
# "Barbara Corcoran"
# "Robert Herjavec"
# "Daymond John"
# "Lori Greiner"
# "Mark Cuban"
# "Others"
shark_to_model = "Deal_Shark 1"

In [13]:
# drop rows where shark_to_model is Nan
data=data.dropna(subset=[shark_to_model])
data2=data2.dropna(subset=[shark_to_model])
data = data.reset_index(drop=True)
data2 = data2.reset_index(drop=True)
data

,Season_Epi_code,Pitched_Business_Identifier,Amount_Asked_For,Exchange_For_Stake,Gender,Category,Pitched_Business_Desc,Deal_Status,Kevin O'Leary,Lori Greiner,Mark Cuban,Robert Herjavec,Daymond John,Barbara Corcoran,Others,Deal_Shark 1,Deal_Shark 2,Deal_Shark 3,Deal_Shark 4,Deal_Shark 5
0,101,Ava the Elephant,50000,0.15,Female,Health / Wellness,"(Emmy the Elephant during show, trademarked a...",1,0,0,0,0,0,1,0,Barbara Corcoran,NaN,NaN,NaN,NaN
1,101,Mr. Tod's Pie Factory,460000,0.10,Male,Food and Beverage,a pie company,1,0,0,0,0,1,1,0,Barbara Corcoran,Daymond John,NaN,NaN,NaN
2,102,A Perfect Pear,500000,0.15,Female,Food and Beverage,a gourmet food business . a gourmet food business,1,0,0,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
3,102,Classroom Jams,250000,0.10,Male,Children / Education,a musical way to teach students Shakespeare . ...,1,1,0,0,1,1,1,0,Barbara Corcoran,Daymond John,Kevin O'Leary,Robert Herjavec,NaN
4,103,Stress Free Kids,250000,0.20,Female,Children / Education,a line of books that use stress relieving tech...,1,0,0,0,0,0,1,0,Barbara Corcoran,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,824,Rumi Spice,250000,0.05,Mixed,Food and Beverage,saffron imported from Afghanistan . Rumi Spice...,1,0,0,1,0,0,0,0,Mark Cuban,NaN,NaN,NaN,NaN
378,824,Thompson Tee,700000,0.07,Male,Fashion / Beauty,undershirts designed to block underarm sweat ....,1,0,0,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
379,824,Wallet Buckle,500000,0.10,Male,Fashion / Beauty,a belt buckle that holds credit cards . The Wa...,1,0,0,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
380,826,Bridal Buddy,75000,0.10,Female,Fashion / Beauty,a functional slip worn under a wedding gown th...,1,1,1,0,0,0,0,0,Kevin O'Leary,Lori Greiner,NaN,NaN,NaN


In [14]:
# Custom tokenizer from class activity 22-1-4
stopwords = stopwords.words( 'english' ) + list(punctuation)
stemmer = PorterStemmer()
# Stemming
punc_list = list(punctuation)
def special_remove(word):
    if len(word)>2:
        return False
    for c in word:
        if c in punc_list:
            return True
    return False

# custom function that overrides default token generation
def custom_tokenizer(text):
    text = text.lower()
    words = nltk.word_tokenize(text)
    words = [stemmer.stem(w) for w in words if w not in stopwords+["..."]]
    # further remove words with a special char
    words = [w for w in words if not special_remove(w)]
    return words

activity_vectorizer = TfidfVectorizer(ngram_range=(1,2)
                            ,tokenizer=custom_tokenizer 
                            ,stop_words='english'
                            # ,min_df=2
                            ,strip_accents='unicode'
                            )

activity_vectors = activity_vectorizer.fit_transform(data["Pitched_Business_Desc"])
activity_df = pd.DataFrame(activity_vectors.toarray(), columns=activity_vectorizer.get_feature_names())
activity_df

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'thu', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


,-not,-not simpli,10,10 day,10 net,"10,000","10,000 run",10-30,10-30 minut,100,...,“ sun,“ sunglass,“ theyr,“ total,“ uv,“ wrap,”,” peopl,” potato,” prevent
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
activity_vectors2 = activity_vectorizer.fit_transform(data2["Pitched_Business_Desc"])
activity_df2 = pd.DataFrame(activity_vectors2.toarray(), columns=activity_vectorizer.get_feature_names())
activity_df2

,-not,-not simpli,10,10 day,10 net,"10,000","10,000 run",10-30,10-30 minut,100,...,“ sun,“ sunglass,“ theyr,“ total,“ uv,“ wrap,”,” peopl,” potato,” prevent
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
activity_vectorizer.get_feature_names()

['-not',
 '-not simpli',
 '10',
 '10 day',
 '10 net',
 '10,000',
 '10,000 run',
 '10-30',
 '10-30 minut',
 '100',
 '100 food',
 '100 guarante',
 '100 mile',
 '100 natur',
 '100 recycl',
 '100 time',
 '100 zinc',
 '12',
 '12 fashion',
 '123',
 '123 plush',
 '15',
 '15 flavor',
 '150',
 '150 90',
 '2',
 '2 ipad',
 '2-5',
 '2-5 age',
 '2-in-1',
 '2-in-1 spray',
 '2012',
 '2012 pipcorn',
 '20mph',
 '20mph 30mph',
 '21st',
 '21st centuri',
 '23',
 '23 ounc',
 '24',
 '24 mph',
 '2400',
 '2400 expert',
 '2400 sat',
 '26',
 '26 pound',
 '26g',
 '26g protein',
 '270',
 '270 shoe',
 '3',
 '3 design',
 '3 ipad',
 '3 pack',
 '3-d',
 '3-d design',
 '3-in-1',
 '3-in-1 nail',
 '30',
 '30 minut',
 '300,000',
 '300,000 fund',
 '30mph',
 '30mph travel',
 '35',
 '35 univers',
 '360',
 '360 degre',
 '360-degre',
 '360-degre pictur',
 '3d',
 '3d pop-up',
 '3d touch-bas',
 '4',
 '40+',
 '40+ mile',
 '5',
 '5 minut',
 '5-in-1',
 '5-in-1 babi',
 '5-in-1 convert',
 '50',
 '50 calib',
 '50 mph',
 '5g',
 '5g fru

In [17]:
# Choose features

# To include Pitch, comment in next line, and comment out the following line
# To exclude Pitch, comment out next line, and comment in the following line

selected_features = activity_df
selected_features2 = activity_df2
# selected_features = pd.DataFrame([])

selected_features["Gender"]=data["Gender"]
selected_features["Category"]=data["Category"]
selected_features["Amount_Asked_For"]=data["Amount_Asked_For"]
selected_features["Exchange_For_Stake"]=data["Exchange_For_Stake"]
selected_features

,-not,-not simpli,10,10 day,10 net,"10,000","10,000 run",10-30,10-30 minut,100,...,“ uv,“ wrap,”,” peopl,” potato,” prevent,Gender,Category,Amount_Asked_For,Exchange_For_Stake
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Female,Health / Wellness,50000,0.15
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Food and Beverage,460000,0.10
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Female,Food and Beverage,500000,0.15
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Children / Education,250000,0.10
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Female,Children / Education,250000,0.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Mixed,Food and Beverage,250000,0.05
378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Fashion / Beauty,700000,0.07
379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Fashion / Beauty,500000,0.10
380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Female,Fashion / Beauty,75000,0.10


In [18]:
selected_features2["Gender"]=data2["Gender"]
selected_features2["Category"]=data2["Category"]
selected_features2["Amount_Asked_For"]=data2["Amount_Asked_For"]
selected_features2["Exchange_For_Stake"]=data2["Exchange_For_Stake"]
selected_features2

,-not,-not simpli,10,10 day,10 net,"10,000","10,000 run",10-30,10-30 minut,100,...,“ uv,“ wrap,”,” peopl,” potato,” prevent,Gender,Category,Amount_Asked_For,Exchange_For_Stake
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Female,Health / Wellness,50000,0.15
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Food and Beverage,460000,0.10
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Food and Beverage,460000,0.10
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Female,Food and Beverage,500000,0.15
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Children / Education,250000,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Fashion / Beauty,700000,0.07
484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Fashion / Beauty,500000,0.10
485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Female,Fashion / Beauty,75000,0.10
486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Female,Fashion / Beauty,75000,0.10


In [19]:
selected_features = pd.get_dummies(selected_features)
selected_features

,-not,-not simpli,10,10 day,10 net,"10,000","10,000 run",10-30,10-30 minut,100,...,Category_Fitness / Sports / Outdoor,Category_Food and Beverage,Category_Green / Clean Tech,Category_Health / Wellness,Category_Lifestyle / Home,Category_Media / Entertainment,Category_Pet Products,Category_Software / Tech,Category_Travel,Category_Uncertain / Other
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
selected_features2 = pd.get_dummies(selected_features2)
selected_features2



,-not,-not simpli,10,10 day,10 net,"10,000","10,000 run",10-30,10-30 minut,100,...,Category_Fitness / Sports / Outdoor,Category_Food and Beverage,Category_Green / Clean Tech,Category_Health / Wellness,Category_Lifestyle / Home,Category_Media / Entertainment,Category_Pet Products,Category_Software / Tech,Category_Travel,Category_Uncertain / Other
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
selected_features.columns

In [38]:
model = MultinomialNB(alpha=.0001)

X_smote, y_smote = SMOTE().fit_sample(selected_features, data[[shark_to_model]].values.ravel())
X = X_smote
y = y_smote

# X = selected_features
# y = data[[shark_to_model]].values.ravel()



print(X.shape)
print(y.shape)

(553, 8653)
(553,)


In [39]:
model2 = MultinomialNB(alpha=.0001)

X_smote2, y_smote2 = SMOTE().fit_sample(selected_features2, data2[[shark_to_model]].values.ravel())
X2 = X_smote2
y2 = y_smote2

# X2 = selected_features2
# y2 = data2[[shark_to_model]].values.ravel()



print(X2.shape)
print(y2.shape)

(707, 8653)
(707,)


In [40]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=f_classif, k=6)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfpvalues = pd.DataFrame(fit.pvalues_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores,dfpvalues],axis=1)
featureScores.columns = ['Specs','Score','PValue']  #naming the dataframe columns
print(featureScores.nlargest(20,'Score'))  #print 10 best features

                          Specs     Score        PValue
7682                      today  9.752111  3.215241e-10
8647  Category_Lifestyle / Home  9.575610  5.026899e-10
600                 bar healthi  9.349620  8.911296e-10
3927                      insid  9.105776  1.653385e-09
8126                     vibrat  9.011536  2.099684e-09
1894                     custom  8.698642  4.643471e-09
3070                      fresh  7.647090  6.696414e-08
7599                      theme  7.622470  7.128095e-08
4395                       live  7.538283  8.825548e-08
1782               creat custom  7.465373  1.061874e-07
3203                       gift  7.456198  1.086881e-07
3752                      human  7.294587  1.637581e-07
503                      awesom  6.863752  4.880065e-07
7919                      ultra  6.845846  5.106460e-07
594                         bar  6.806298  5.644341e-07
8091                      vegan  6.728201  6.878293e-07
199                      afford  6.482840  1.279

In [41]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=6)
fit = bestfeatures.fit(X2,y2)
dfscores = pd.DataFrame(fit.scores_)
dfpvalues = pd.DataFrame(fit.pvalues_)
dfcolumns = pd.DataFrame(X2.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores,dfpvalues],axis=1)
featureScores.columns = ['Specs','Score','PValue']  #naming the dataframe columns
print(featureScores.nlargest(20,'Score'))  #print 10 best features

                           Specs         Score        PValue
8634            Amount_Asked_For  2.222600e+07  0.000000e+00
8647   Category_Lifestyle / Home  5.821176e+01  1.038072e-10
8640  Category_Business Services  4.200000e+01  1.838771e-07
8636               Gender_Female  3.716556e+01  1.634746e-06
8644  Category_Food and Beverage  3.132110e+01  2.200999e-05
8642   Category_Fashion / Beauty  2.037383e+01  2.375340e-03
990                   breathalyz  1.749797e+01  7.617251e-03
6867        smartphon breathalyz  1.749797e+01  7.617251e-03
8650    Category_Software / Tech  1.706977e+01  9.030230e-03
8646  Category_Health / Wellness  1.681818e+01  9.975200e-03
594                          bar  1.621826e+01  1.262922e-02
8637                 Gender_Male  1.513990e+01  1.919610e-02
4855                        moss  1.477860e+01  2.205042e-02
2136                 devic water  1.474778e+01  2.231183e-02
7838                 tree effici  1.474778e+01  2.231183e-02
8243                  wa

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [43]:
model.fit(X_train, y_train.ravel())

MultinomialNB(alpha=0.0001, class_prior=None, fit_prior=True)

In [44]:
y_pred = model.predict(X_test)

In [45]:
pd.DataFrame({"actual": y_test.reshape(-1), "prediction": y_pred.reshape(-1)})

,actual,prediction
0,Robert Herjavec,Robert Herjavec
1,Daymond John,Barbara Corcoran
2,Robert Herjavec,Robert Herjavec
3,Kevin O'Leary,Kevin O'Leary
4,Mark Cuban,Mark Cuban
...,...,...
134,Kevin O'Leary,Kevin O'Leary
135,Barbara Corcoran,Barbara Corcoran
136,Daymond John,Others
137,Daymond John,Daymond John


In [46]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.697841726618705

In [47]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, random_state=42)

In [48]:
model2.fit(X_train2, y_train2.ravel())

MultinomialNB(alpha=0.0001, class_prior=None, fit_prior=True)

In [49]:
y_pred2 = model2.predict(X_test2)

In [50]:
pd.DataFrame({"actual": y_test2.reshape(-1), "prediction": y_pred2.reshape(-1)})

,actual,prediction
0,Barbara Corcoran,Lori Greiner
1,Lori Greiner,Lori Greiner
2,Lori Greiner,Robert Herjavec
3,Robert Herjavec,Robert Herjavec
4,Others,Others
...,...,...
172,Lori Greiner,Lori Greiner
173,Others,Others
174,Kevin O'Leary,Kevin O'Leary
175,Barbara Corcoran,Barbara Corcoran


In [51]:
accuracy_score(y_test2,y_pred2)

0.7570621468926554

In [36]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
param_grid = {'alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}
scores = ['precision', 'recall']
model3 = MultinomialNB()

In [37]:
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        model3, param_grid, scoring='%s_macro' % score,verbose=3)
    clf.fit(X_train2, y_train2)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test2, clf.predict(X_test2)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] alpha=1e-05 .....................................................
[CV] ......................... alpha=1e-05, score=0.725, total=   0.1s
[CV] alpha=1e-05 .....................................................
[CV] ......................... alpha=1e-05, score=0.778, total=   0.1s
[CV] alpha=1e-05 .....................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV] ......................... alpha=1e-05, score=0.770, total=   0.1s
[CV] alpha=1e-05 .....................................................
[CV] ......................... alpha=1e-05, score=0.792, total=   0.1s
[CV] alpha=1e-05 .....................................................
[CV] ......................... alpha=1e-05, score=0.822, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.736, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.769, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.770, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.812, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] .

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[CV] ............................. alpha=1, score=0.022, total=   0.1s
[CV] alpha=1 .........................................................
[CV] ............................. alpha=1, score=0.020, total=   0.1s
[CV] alpha=1 .........................................................
[CV] ............................. alpha=1, score=0.020, total=   0.1s
[CV] alpha=1 .........................................................

...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    3.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ........................... alpha=100, score=0.022, total=   0.1s
Best parameters set found on development set:

{'alpha': 0.0001}

Grid scores on development set:

0.777 (+/-0.063) for {'alpha': 1e-05}
0.779 (+/-0.056) for {'alpha': 0.0001}
0.770 (+/-0.067) for {'alpha': 0.001}
0.768 (+/-0.055) for {'alpha': 0.01}
0.675 (+/-0.093) for {'alpha': 0.1}
0.031 (+/-0.042) for {'alpha': 1}
0.021 (+/-0.001) for {'alpha': 10}
0.021 (+/-0.001) for {'alpha': 100}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

                  precision    recall  f1-score   support

Barbara Corcoran       0.59      0.70      0.64        23
    Daymond John       0.79      0.62      0.70        24
   Kevin O'Leary       0.83      0.76      0.79        25
    Lori Greiner       0.61      0.53      0.57        32
      Mark Cuban       0.76      0.79      0.78        24
          Others       0.78      1.00      0.88    

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV] ......................... alpha=1e-05, score=0.799, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.733, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.759, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.773, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.810, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.790, total=   0.1s
[CV] alpha=0.001 .....................................................
[CV] ......................... alpha=0.001, score=0.724, total=   0.1s
[CV] alpha=0.001 .....................................................
[CV] .

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    3.3s finished


In [52]:
# Import ML models from sklearn
from sklearn.linear_model import LogisticRegression # Regression classifier
from sklearn.tree import DecisionTreeClassifier # Decision Tree classifier
from sklearn import svm # Support Vector Machine
from sklearn.linear_model import SGDClassifier # Stochastic Gradient Descent Classifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier # Random Forest and Gradient Boosting Classifier
from sklearn.naive_bayes import MultinomialNB # Naive Bayes Classifier 
from sklearn.metrics import f1_score, accuracy_score, recall_score, confusion_matrix # Some metrics to check the performance of the models

In [53]:
# Setting parameters for each algorithm - these are tunable to achieve max accuracy

Classifiers = {'LR':LogisticRegression(random_state=10,C=5,max_iter=300, solver='lbfgs')
               ,'DTC':DecisionTreeClassifier(random_state=10,min_samples_leaf=2)
               ,'RF':RandomForestClassifier(random_state=10,n_estimators=100,n_jobs=-1)
               # ,'GBC':GradientBoostingClassifier(random_state=10,n_estimators=400,learning_rate=0.2)
               ,'SGD':SGDClassifier(loss="hinge", penalty="l2")
               # ,'SVM':svm.SVC(kernel='linear', C=0.1)
               ,'NB':MultinomialNB(alpha=.0001)
               }

In [54]:
Classifiers.keys()

dict_keys(['LR', 'DTC', 'RF', 'SGD', 'NB'])

In [55]:
# Create a pipeline so you can reuse the code

def ML_Pipeline(clf_name):
    clf = Classifiers[clf_name]
    fit = clf.fit(X_train, y_train.ravel())
    y_pred = clf.predict(X_test)
    Accuracy = accuracy_score(y_test,y_pred)
    Precision = metrics.precision_score(y_test,y_pred)
    Recall = recall_score(y_test,y_pred)
    Confusion_matrix = confusion_matrix(y_test,y_pred)
    print('Classifier = '+(clf_name))
    print('Accuracy = '+str(Accuracy))
    print('Precision = '+str(Precision))
    print('Recall = '+str(Recall))
    print(Confusion_matrix)
    print('==='*20) 
    return clf

In [73]:
# Create a pipeline so you can reuse the code

def ML_Pipeline2(clf_name):
    clf = Classifiers[clf_name]
    fit = clf.fit(X_train2, y_train2.ravel())
    y_pred = clf.predict(X_test2)
    F1 = f1_score(y_test2, y_pred, zero_division=1,average="weighted")
    Accuracy = accuracy_score(y_test2,y_pred)
    Precision = metrics.precision_score(y_test2,y_pred,average="weighted",zero_division=1)
    Recall = recall_score(y_test2,y_pred,average="weighted")
    Confusion_matrix = confusion_matrix(y_test2,y_pred)
    print('Classifier = '+(clf_name))
    print('F1 = '+str(F1))
    print('Accuracy = '+str(Accuracy))
    print('Precision = '+str(Precision))
    print('Recall = '+str(Recall))
    print(Confusion_matrix)
    print('==='*20) 
    return clf

In [67]:
clf_dict = []
for clf_name in Classifiers.keys():
    clf = Classifiers[clf_name]
    fit = clf.fit(X_train, y_train.ravel())
    y_pred = clf.predict(X_test)
    Accuracy = accuracy_score(y_test,y_pred)
    Precision = metrics.precision_score(y_test,y_pred,average="weighted",zero_division=1)
    Recall = recall_score(y_test,y_pred,average="weighted")
    Confusion_matrix = confusion_matrix(y_test,y_pred)
    clf_dict.append({'Classifier': clf_name, 'Accuracy': Accuracy, 'Precision': Precision, 'Recall': Recall})
    print('Classifier = '+(clf_name))
    print('Accuracy = '+str(Accuracy))
    print('Precision = '+str(Precision))
    print('Recall = '+str(Recall))
    print(Confusion_matrix)
    print('==='*20)


Classifier = LR
Accuracy = 0.10071942446043165
Precision = 0.9094249780032089
Recall = 0.10071942446043165
[[ 0  0  0  0 22  0  0]
 [ 0  0  0  0 27  0  0]
 [ 0  0  0  0 15  0  0]
 [ 0  0  0  0 21  0  0]
 [ 0  0  0  0 14  0  0]
 [ 0  0  0  0 20  0  0]
 [ 0  0  0  0 20  0  0]]
Classifier = DTC
Accuracy = 0.39568345323741005
Precision = 0.4555330992123297
Recall = 0.39568345323741005
[[11  1  2  2  5  0  1]
 [ 5  8  2  5  4  1  2]
 [ 7  1  5  0  2  0  0]
 [ 4  7  1  5  3  0  1]
 [ 0  4  2  4  4  0  0]
 [ 3  2  0  3  2 10  0]
 [ 2  2  0  1  3  0 12]]
Classifier = RF
Accuracy = 0.60431654676259
Precision = 0.7070508103789305
Recall = 0.60431654676259
[[18  0  1  1  2  0  0]
 [12  6  0  6  2  0  1]
 [ 4  0 10  1  0  0  0]
 [ 6  1  0 12  2  0  0]
 [ 7  0  1  1  5  0  0]
 [ 0  0  1  2  1 16  0]
 [ 3  0  0  0  0  0 17]]
Classifier = SGD
Accuracy = 0.1510791366906475
Precision = 0.871745768852544
Recall = 0.1510791366906475
[[ 0  0  0 22  0  0  0]
 [ 0  0  0 27  0  0  0]
 [ 0  0  0 15  0  0  0]


In [68]:
clf_dict2 = []
for clf_name in Classifiers.keys():
    clf = Classifiers[clf_name]
    fit = clf.fit(X_train2, y_train2.ravel())
    y_pred = clf.predict(X_test2)
    F1 = f1_score(y_test2, y_pred, zero_division=1,average="weighted")
    Accuracy = accuracy_score(y_test2,y_pred)
    Precision = metrics.precision_score(y_test2,y_pred,average="weighted",zero_division=1)
    Recall = recall_score(y_test2,y_pred,average="weighted")
    Confusion_matrix = confusion_matrix(y_test2,y_pred)
    clf_dict2.append({'Classifier': clf_name, 'F1': F1, 'Accuracy': Accuracy, 'Precision': Precision, 'Recall': Recall})
    print('Classifier = '+(clf_name))
    print('F1 = '+str(F1))
    print('Accuracy = '+str(Accuracy))
    print('Precision = '+str(Precision))
    print('Recall = '+str(Recall))
    print(Confusion_matrix)
    print('==='*20)

Classifier = LR
F1 = 0.03238047052871237
Accuracy = 0.13559322033898305
Precision = 0.882792301062913
Recall = 0.13559322033898305
[[ 0  0  0  0 23  0  0]
 [ 0  0  0  0 24  0  0]
 [ 0  0  0  0 25  0  0]
 [ 0  0  0  0 32  0  0]
 [ 0  0  0  0 24  0  0]
 [ 0  0  0  0 21  0  0]
 [ 0  0  0  0 28  0  0]]
Classifier = DTC
F1 = 0.5214926653205615
Accuracy = 0.5141242937853108
Precision = 0.5928546246928499
Recall = 0.5141242937853108
[[12  5  2  2  1  1  0]
 [ 8 13  1  0  1  1  0]
 [ 4  3 17  0  1  0  0]
 [11  7  4  8  2  0  0]
 [ 3  4  0  1 14  1  1]
 [ 1  3  2  0  0 15  0]
 [ 2  5  1  4  2  2 12]]
Classifier = RF
F1 = 0.6548468741075402
Accuracy = 0.6384180790960452
Precision = 0.7174341610633461
Recall = 0.6384180790960452
[[19  2  0  2  0  0  0]
 [ 6 13  0  3  1  0  1]
 [ 6  2 16  0  1  0  0]
 [10  7  1 12  2  0  0]
 [ 5  4  0  0 15  0  0]
 [ 2  1  0  1  0 17  0]
 [ 3  0  0  4  0  0 21]]
Classifier = SGD
F1 = 0.043213449083643385
Accuracy = 0.15819209039548024
Precision = 0.866832647068211

In [69]:
clf_df = pd.DataFrame(clf_dict2)
clf_df

,Classifier,F1,Accuracy,Precision,Recall
0,LR,0.032380,0.135593,0.882792,0.135593
1,DTC,0.521493,0.514124,0.592855,0.514124
2,RF,0.654847,0.638418,0.717434,0.638418
3,SGD,0.043213,0.158192,0.866833,0.158192
4,NB,0.751344,0.757062,0.755338,0.757062


In [70]:
clf_df['Accuracy'].max()

0.7570621468926554

In [71]:
best_clf = clf_df.loc[clf_df['Accuracy'].idxmax(),'Classifier']
best_clf

'NB'

In [74]:
best_model = ML_Pipeline2(best_clf)
best_model


Classifier = NB
F1 = 0.7513442855618668
Accuracy = 0.7570621468926554
Precision = 0.7553378678372056
Recall = 0.7570621468926554
[[14  2  0  5  1  1  0]
 [ 2 15  2  2  1  1  1]
 [ 3  1 18  2  1  0  0]
 [ 4  1  1 18  4  1  3]
 [ 1  0  0  0 22  0  1]
 [ 0  0  0  0  0 21  0]
 [ 0  0  0  1  0  1 26]]


MultinomialNB(alpha=0.0001, class_prior=None, fit_prior=True)

In [77]:
import joblib
joblib.dump(best_model, f"{shark_to_model}_model.pkl")

['Deal_Shark 1_model.pkl']

In [78]:
import pickle
pickle.dump(selected_features.columns,open(f"{shark_to_model}_vocab.pkl","wb"))